In [1]:
from google_play_scraper import app, reviews
import pandas as pd
import warnings
import json
from app_store_scraper import AppStore
warnings.filterwarnings('ignore')

### Google

In [2]:
# %%time

# import itertools
# import string
# from google_play_scraper import app, Sort, reviews, reviews_all
# from app_store_scraper import AppStore
# import pandas as pd
# import numpy as nnp
# import json, os, uuid

# reviewCount = 1500
# country = 'sg'
# language = 'en'

# # Define the set of characters to use (excluding uppercase letters)
# characters = string.digits + string.ascii_lowercase + string.punctuation + ' '

# # Define the minimum and maximum length of the string
# min_length = 3 # 25
# max_length = 150 # 25

# google_main = pd.DataFrame(columns = ['title', 'description', 'descriptionHTML',
#                         'summary', 'installs', 'minInstalls', 'realInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'currency', 'sale',
#                         'saleTime', 'originalPrice', 'saleText', 'offersIAP', 'inAppProductPrice', 'developer', 'developerId', 'developerAddress', 'genre',
#                         'genreId', 'categories', 'contentRating', 'contentRatingDescription', 'adSupported', 'containsAds', 'released',
#                         'lastUpdatedOn', 'updated', 'version', 'appId'])

# google_reviews = pd.DataFrame(columns = ['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
#                             'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'appId'])

# mainFeaturesToDrop = ['url', 'comments', 'video', 'videoImage', 'screenshots', 'headerImage', 'icon', 'privacyPolicy', 'developerWebsite', 'developerEmail']
# reviewFeaturesToDrop = ['userImage']

# reviewCountRange = range(0,reviewCount)

# # Loop over all possible lengths of the string
# for length in range(min_length, max_length + 1):
#     # Generate all combinations of characters of the current length
#     for combination in itertools.product(characters, repeat=length):
#         # Convert the combination to a string
#         appId = ''.join(combination)

#         try:
#             app_results = app(
#             appId,
#             lang=language, # defaults to 'en'
#             country=country # defaults to 'us'
#             )

#             for feature in mainFeaturesToDrop:
#                 app_results.pop(feature, None)
            
#             row = [value for value in app_results.values()]
#             google_main.loc[len(google_main)] = row

#             for score in range(1,6):

#                 review, continuation_token = reviews(
#                     appId,
#                     lang=language, # defaults to 'en'
#                     country=country, # defaults to 'us'
#                     sort=Sort.NEWEST, # defaults to Sort.NEWEST
#                     count=reviewCount, # defaults to 100
#                     filter_score_with=score # defaults to None(means all score)
#                 )

#                 for count in reviewCountRange:

#                     try:
#                         for feature in reviewFeaturesToDrop:
#                             review[count].pop(feature, None)
#                         row = [value for value in review[count].values()]
#                         row.append(appId)
#                         google_reviews.loc[len(google_reviews)] = row
#                     except IndexError:
#                         continue
            
#             with open("FullGoogleDataIngestion.log", "a") as log_file:
#                 log_file.write(f"{appId} .. Success: {len(google_main)} app(s) saved into google_main. {len(google_reviews)} review(s) saved in google_reviews.\n")
            
#         except Exception as e:
#             with open("FullGoogleDataIngestion.log", "a") as log_file:
#                 log_file.write(f"{appId} .. Error occurred: {e}\n")

In [34]:
# Import libraries and datasets
import os
import subprocess
import glob
import shutil
import pandas as pd
import json
from google.cloud import bigquery
from pandas_gbq import read_gbq
from datetime import datetime
from pytz import timezone

from google_play_scraper import app, reviews, Sort
import warnings
# from app_store_scraper import AppStore
warnings.filterwarnings('ignore')

folder_path = os.getcwd().replace("\\", "/")

# Hard-coded variables
project_id = "big-data-analytics-412816"
rawDataset = "practice_project"
# apple_db_path = f"{project_id}.{rawDataset}.apple_scraped"
googleScraped_db_path = f"{project_id}.{rawDataset}.google_scraped"
googleReview_db_path = f"{project_id}.{rawDataset}.google_review"
# dateTime_db_path = f"{project_id}.{rawDataset}.dateTime"
googleScraped_csv_path = f"{folder_path}/google_scraped.csv"
googleReview_csv_path = f"{folder_path}/google_review.csv"
# dateTime_csv_path = f"{folder_path}/dateTime.csv"
googleAPI_json_path = f"{folder_path}/big-data-analytics-412816-1be796546c90.json"

client = bigquery.Client.from_service_account_json(googleAPI_json_path)

##############################

google = pd.read_csv('C:/Users/Michael/Documents/GitHub/appStoreAnalytics/dataSources/google.csv')

reviewCount = 100
country = 'us'
language = 'en'

google_main = pd.DataFrame(columns = ['title', 'description', 'descriptionHTML',
                        'summary', 'installs', 'minInstalls', 'realInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'currency', 'sale',
                        'saleTime', 'originalPrice', 'saleText', 'offersIAP', 'inAppProductPrice', 'developer', 'developerId', 'developerAddress', 'genre',
                        'genreId', 'categories', 'contentRating', 'contentRatingDescription', 'adSupported', 'containsAds', 'released',
                        'lastUpdatedOn', 'updated', 'version', 'appId'])

google_reviews = pd.DataFrame(columns = ['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
                            'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'appId'])

mainFeaturesToDrop = ['url', 'comments', 'video', 'videoImage', 'screenshots', 'headerImage', 'icon', 'privacyPolicy', 'developerWebsite', 'developerEmail']
reviewFeaturesToDrop = ['userImage']

reviewCountRange = range(0,reviewCount)

for appId in google['App Id'].sample(100).unique():

    appReivewCounts = 0

    try:
        app_results = app(
        appId,
        lang=language, # defaults to 'en'
        country=country # defaults to 'us'
        )

        for feature in mainFeaturesToDrop:
            app_results.pop(feature, None)
        
        row = [value for value in app_results.values()]
        google_main.loc[len(google_main)] = row

        for score in range(1,6):

            review, continuation_token = reviews(
                appId,
                lang=language, # defaults to 'en'
                country=country, # defaults to 'us'
                sort=Sort.NEWEST, # defaults to Sort.NEWEST
                count=reviewCount, # defaults to 100
                filter_score_with=score # defaults to None(means all score)
            )

            for count in reviewCountRange:

                try:
                    for feature in reviewFeaturesToDrop:
                        review[count].pop(feature, None)
                    row = [value for value in review[count].values()]
                    row.append(appId)
                    google_reviews.loc[len(google_reviews)] = row
                    appReivewCounts += 1
                except IndexError:
                    continue
        
        with open("googleDataIngestion.log", "a") as log_file:
            log_file.write(f"{appId} -> Successfully saved with {appReivewCounts} reviews. Total: {len(google_main)} app(s) & {len(google_reviews)} review(s) saved.\n")
        
    except Exception as e:
        # with open("googleDataIngestion.log", "a") as log_file:
        #     log_file.write(f"{appId} .. Error occurred: {e}\n")
        pass

try:
    job = client.query(f"DELETE FROM {googleScraped_db_path} WHERE TRUE").result()
except:
    pass
client.create_table(bigquery.Table(googleScraped_db_path), exists_ok = True)
try:
    job = client.query(f"DELETE FROM {googleReview_db_path} WHERE TRUE").result()
except:
    pass
client.create_table(bigquery.Table(googleReview_db_path), exists_ok = True)

google_main = google_main.astype(str) # all columns will be string!!
google_main.to_gbq(destination_table=googleScraped_db_path, project_id=project_id, if_exists='replace')
google_reviews.to_gbq(destination_table=googleReview_db_path, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [31]:
google_main = google_main.astype(str)

In [33]:
google_main.to_gbq(destination_table=googleScraped_db_path, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [19]:
google_main['categories'] = google_main['categories'].astype(str)

In [18]:
google_main.shape

(33, 35)

In [17]:
google_main.head()

title  \
0                  Course Videos   
1  Prayer Time & Qibla Direction   
2            Skyscraperarchitect   
3                 89.7 The River   
4                    InstantWizz   

                                         description  \
0  Premium Online and Offline Access to the World...   
1  Prayer Time and Qibla Direction, It is a smart...   
2  Best Architect Apps or Vastu Science Consultin...   
3  The Cutting Edge of Rock\r\n89.7 The River (KI...   
4  Have you ever struggled with formatting the ca...   

                                     descriptionHTML  \
0  Premium Online and Offline Access to the World...   
1  Prayer Time and Qibla Direction, It is a smart...   
2  Best Architect Apps or Vastu Science Consultin...   
3  The Cutting Edge of Rock<br>89.7 The River (KI...   
4  Have you ever struggled with formatting the ca...   

                                             summary installs  minInstalls  \
0  Online and Offline Access to the Leadership In...      10+           10   
1  Simple and stylish free app designed to find P...  50,000+        50000   
2            Welcome to our Skyscraperarchitect app.      50+           50   
3                     89.7 The River Streaming Radio   5,000+         5000   
4        Captions and hashtags manager for Instagram     100+          100   

   realInstalls  score  ratings  reviews  ...          genreId  contentRating  \
0            38   0.00        0        0  ...        EDUCATION       Everyone   
1         53745   0.00        0        0  ...        LIFESTYLE       Everyone   
2            83   5.00       32        0  ...         BUSINESS       Everyone   
3          6619   4.56      104       53  ...  MUSIC_AND_AUDIO           Teen   
4           148   0.00        0        0  ...     PRODUCTIVITY       Everyone   

              contentRatingDescription adSupported  containsAds      released  \
0                                 None       False        False  Nov 21, 2018   
1                                 None        True         True  Oct 29, 2015   
2                                 None       False        False   Sep 6, 2017   
3  Diverse Content: Discretion Advised        True         True  Aug 14, 2017   
4                                 None       False        False   Jan 7, 2020   

  lastUpdatedOn     updated  version                                appId  
0  Jul 26, 2020  1595831874    2.1.0  com.release.lifestartupsfacilitator  
1  Aug 24, 2023  1692867424    5.0.0            com.ustadmob.qiblacompass  
2   Jan 4, 2018  1515076149    1.1.0      com.appswiz.skyscraperarchitect  
3  Dec 21, 2023  1703194257    1.4.3               edu.iwcc.apps.theriver  
4   Mar 5, 2021  1614974989    3.5.3                        app.instawizz  

[5 rows x 34 columns]

In [ ]:
# Import libraries and datasets
import os
import subprocess
import glob
import shutil
import pandas as pd
import json
from google.cloud import bigquery
from pandas_gbq import read_gbq
from datetime import datetime
from pytz import timezone

from google_play_scraper import app, reviews, Sort
import warnings
# from app_store_scraper import AppStore
warnings.filterwarnings('ignore')

folder_path = os.getcwd().replace("\\", "/")

# Hard-coded variables
project_id = "big-data-analytics-412816"
rawDataset = "practice_project"
# apple_db_path = f"{project_id}.{rawDataset}.apple_scraped"
googleScraped_db_path = f"{project_id}.{rawDataset}.google_scraped"
googleReview_db_path = f"{project_id}.{rawDataset}.google_review"
# dateTime_db_path = f"{project_id}.{rawDataset}.dateTime"
googleScraped_csv_path = f"{folder_path}/google_scraped.csv"
googleReview_csv_path = f"{folder_path}/google_review.csv"
# dateTime_csv_path = f"{folder_path}/dateTime.csv"
googleAPI_json_path = f"{folder_path}/big-data-analytics-412816-1be796546c90.json"

client = bigquery.Client.from_service_account_json(googleAPI_json_path)

##############################

google = pd.read_csv('C:/Users/Michael/Documents/GitHub/appStoreAnalytics/dataSources/google.csv')

reviewCount = 10
country = 'us'
language = 'en'

google_main = pd.DataFrame(columns = ['title', 'description', 'descriptionHTML',
                        'summary', 'installs', 'minInstalls', 'realInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'currency', 'sale',
                        'saleTime', 'originalPrice', 'saleText', 'offersIAP', 'inAppProductPrice', 'developer', 'developerId', 'developerAddress', 'genre',
                        'genreId', 'categories', 'contentRating', 'contentRatingDescription', 'adSupported', 'containsAds', 'released',
                        'lastUpdatedOn', 'updated', 'version', 'appId'])

google_reviews = pd.DataFrame(columns = ['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
                            'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'appId'])

mainFeaturesToDrop = ['url', 'comments', 'video', 'videoImage', 'screenshots', 'headerImage', 'icon', 'privacyPolicy', 'developerWebsite', 'developerEmail']
reviewFeaturesToDrop = ['userImage']

reviewCountRange = range(0,reviewCount)

for appId in google['App Id'].sample(50).unique():

    try:
        app_results = app(
        appId,
        lang=language, # defaults to 'en'
        country=country # defaults to 'us'
        )

        for feature in mainFeaturesToDrop:
            app_results.pop(feature, None)
        
        row = [value for value in app_results.values()]
        google_main.loc[len(google_main)] = row

        for score in range(1,6):

            review, continuation_token = reviews(
                appId,
                lang=language, # defaults to 'en'
                country=country, # defaults to 'us'
                sort=Sort.NEWEST, # defaults to Sort.NEWEST
                count=reviewCount, # defaults to 100
                filter_score_with=score # defaults to None(means all score)
            )

            for count in reviewCountRange:

                try:
                    for feature in reviewFeaturesToDrop:
                        review[count].pop(feature, None)
                    row = [value for value in review[count].values()]
                    row.append(appId)
                    google_reviews.loc[len(google_reviews)] = row
                except IndexError:
                    continue
        
        with open("googleDataIngestion.log", "a") as log_file:
            log_file.write(f"{appId} .. Success: {len(google_main)} app(s) saved into google_main. {len(google_reviews)} review(s) saved in google_reviews.\n")
        
    except Exception as e:
        with open("googleDataIngestion.log", "a") as log_file:
            log_file.write(f"{appId} .. Error occurred: {e}\n")

google_main.to_csv(googleScraped_csv_path, header = True, index = False)
google_reviews.to_csv(googleReview_csv_path, header = True, index = False)

##############################

# Extract Google API from GitHub Secret Variable
# googleAPI_dict = json.loads(os.environ["GOOGLEAPI"])
# with open("googleAPI.json", "w") as f:
#     json.dump(googleAPI_dict, f)

# Create tables into Google BigQuery
## Create 'apple' table in DB
# job = client.query(f"DELETE FROM {apple_db_path} WHERE TRUE").result()
# client.create_table(bigquery.Table(apple_db_path), exists_ok = True)
## Create 'google' table in DB
# job = client.query(f"DELETE FROM {google_db_path} WHERE TRUE").result()
client.create_table(bigquery.Table(googleScraped_db_path), exists_ok = True)
client.create_table(bigquery.Table(googleReview_db_path), exists_ok = True)

# Save data as CSV files
# apple.columns = [name.replace(" ", "_") for name in apple.columns]
# apple.to_csv(apple_csv_path, header = True, index = False)
# google.columns = [name.replace(" ", "_") for name in google.columns]
# google.to_csv(google_csv_path, header = True, index = False)

# Push data into DB
# apple_job_config = bigquery.LoadJobConfig(
#     autodetect=True,
#     max_bad_records=5,
#     source_format=bigquery.SourceFormat.CSV
# )
# apple_config = client.dataset(rawDataset).table('apple_scraped')
# with open(apple_csv_path, 'rb') as f:
#     apple_load_job = client.load_table_from_file(f, apple_config, job_config=apple_job_config)
# apple_load_job.result()

googleScraped_job_config = bigquery.LoadJobConfig(
    autodetect=True,
    max_bad_records=5,
    source_format=bigquery.SourceFormat.CSV
)
google_scraped_config = client.dataset(rawDataset).table('google_scraped')
with open(googleScraped_csv_path, 'rb') as f:
    googleScraped_load_job = client.load_table_from_file(f, google_scraped_config, job_config=googleScraped_job_config)
googleScraped_load_job.result()

googleReview_job_config = bigquery.LoadJobConfig(
    autodetect=True,
    max_bad_records=5,
    source_format=bigquery.SourceFormat.CSV
)
google_review_config = client.dataset(rawDataset).table('google_review')
with open(googleReview_csv_path, 'rb') as f:
    googleReview_load_job = client.load_table_from_file(f, google_review_config, job_config=googleReview_job_config)
googleReview_load_job.result()

# # Create 'dateTime' table in DB
# job = client.query(f"DELETE FROM {dateTime_db_path} WHERE TRUE").result()
# client.create_table(bigquery.Table(dateTime_db_path), exists_ok = True)
# current_time = datetime.now(timezone('Asia/Shanghai'))
# timestamp_string = current_time.isoformat()
# dt = datetime.strptime(timestamp_string, '%Y-%m-%dT%H:%M:%S.%f%z')
# date_time_str = dt.strftime('%d-%m-%Y %H:%M:%S')
# time_zone = dt.strftime('%z')
# output = f"{date_time_str}; GMT+{time_zone[2]} (SGT)"
# dateTime_df = pd.DataFrame(data = [output], columns = ['dateTime'])
# dateTime_df.to_csv(dateTime_csv_path, header = True, index = False)
# dateTime_job_config = bigquery.LoadJobConfig(
#     autodetect=True,
#     skip_leading_rows=1,
#     source_format=bigquery.SourceFormat.CSV,
# )
# dateTime_config = client.dataset(rawDataset).table('dateTime')
# with open(dateTime_csv_path, 'rb') as f:
#     dateTime_load_job = client.load_table_from_file(f, dateTime_config, job_config=dateTime_job_config)
# dateTime_load_job.result()

# ## Remove CSV files and folder
# try:
#     os.remove(apple_csv_path)
#     os.remove(google_csv_path)
#     os.remove(dateTime_csv_path)
#     os.remove(googleAPI_json_path)
#     shutil.rmtree(f"{folder_path}apple-appstore-apps")
# except:
#     pass

# apple_query = f"""
#     SELECT *
#     FROM {apple_db_path}
# """
# apple_df = read_gbq(apple_query, project_id)

# google_query = f"""
#     SELECT *
#     FROM {google_db_path}
# """
# google_df = read_gbq(google_query, project_id)

In [ ]:
%%time

google = pd.read_csv('C:/Users/Michael/Documents/GitHub/appStoreAnalytics/dataSources/google.csv')

reviewCount = 300
country = 'us'
language = 'en'

google_main = pd.DataFrame(columns = ['title', 'description', 'descriptionHTML',
                        'summary', 'installs', 'minInstalls', 'realInstalls', 'score', 'ratings', 'reviews', 'histogram', 'price', 'free', 'currency', 'sale',
                        'saleTime', 'originalPrice', 'saleText', 'offersIAP', 'inAppProductPrice', 'developer', 'developerId', 'developerAddress', 'genre',
                        'genreId', 'categories', 'contentRating', 'contentRatingDescription', 'adSupported', 'containsAds', 'released',
                        'lastUpdatedOn', 'updated', 'version', 'appId'])

google_reviews = pd.DataFrame(columns = ['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
                            'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'appId'])

mainFeaturesToDrop = ['url', 'comments', 'video', 'videoImage', 'screenshots', 'headerImage', 'icon', 'privacyPolicy', 'developerWebsite', 'developerEmail']
reviewFeaturesToDrop = ['userImage']

reviewCountRange = range(0,reviewCount)

for appId in google['App Id'].unique():

    try:
        app_results = app(
        appId,
        lang=language, # defaults to 'en'
        country=country # defaults to 'us'
        )

        for feature in mainFeaturesToDrop:
            app_results.pop(feature, None)
        
        row = [value for value in app_results.values()]
        google_main.loc[len(google_main)] = row

        for score in range(1,6):

            review, continuation_token = reviews(
                appId,
                lang=language, # defaults to 'en'
                country=country, # defaults to 'us'
                sort=Sort.NEWEST, # defaults to Sort.NEWEST
                count=reviewCount, # defaults to 100
                filter_score_with=score # defaults to None(means all score)
            )

            for count in reviewCountRange:

                try:
                    for feature in reviewFeaturesToDrop:
                        review[count].pop(feature, None)
                    row = [value for value in review[count].values()]
                    row.append(appId)
                    google_reviews.loc[len(google_reviews)] = row
                except IndexError:
                    continue
        
        with open("googleDataIngestion.log", "a") as log_file:
            log_file.write(f"{appId} .. Success: {len(google_main)} app(s) saved into google_main. {len(google_reviews)} review(s) saved in google_reviews.\n")
        
    except Exception as e:
        with open("googleDataIngestion.log", "a") as log_file:
            log_file.write(f"{appId} .. Error occurred: {e}\n")

### Apple

In [ ]:
slack = AppStore(country='us', app_name='slack', app_id = '618783545')

slack.review(how_many=2000)
slack.reviews

2024-02-12 00:27:40,264 [INFO] Base - Initialised: AppStore('us', 'slack', 618783545)
2024-02-12 00:27:40,264 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/slack/id618783545
2024-02-12 00:27:45,785 [INFO] Base - [id:618783545] Fetched 240 reviews (240 fetched in total)
2024-02-12 00:27:51,637 [INFO] Base - [id:618783545] Fetched 500 reviews (500 fetched in total)
2024-02-12 00:27:57,718 [INFO] Base - [id:618783545] Fetched 760 reviews (760 fetched in total)
2024-02-12 00:28:03,458 [INFO] Base - [id:618783545] Fetched 1020 reviews (1020 fetched in total)
2024-02-12 00:28:09,196 [INFO] Base - [id:618783545] Fetched 1240 reviews (1240 fetched in total)
2024-02-12 00:28:14,841 [INFO] Base - [id:618783545] Fetched 1520 reviews (1520 fetched in total)
2024-02-12 00:28:20,261 [INFO] Base - [id:618783545] Fetched 1800 reviews (1800 fetched in total)
2024-02-12 00:28:24,047 [INFO] Base - [id:618783545] Fetched 2000 reviews (2000 fetched in total)


[{'date': datetime.datetime(2021, 11, 19, 15, 37, 47),
  'developerResponse': {'id': 26382584,
   'body': "Hey there, thank you for the helpful feedback! We're sorry to hear you're having a tough time with notifications. Could you please write to us at feedback@slack.com? We may be able to investigate the issue for you and help!",
   'modified': '2021-11-20T23:14:05Z'},
  'review': 'The app is easy to learn and has become an integral part of our small business. \nI wish I could put the channels in the order we use them the most, instead of the app just putting them where it wants. \nI had an issue with posting pics and tech help was friendly and got back with me quickly. The problem was resolved in a week or so. \nMy biggest complaint is the app is bad about not giving notifications when it should. This is a real problem as many employees will not check the app unless it notifies them something has been posted. Every time there is an update, the notifications seem to work well for a fe